<a href="https://colab.research.google.com/github/imcalledaditi/Dynamic-Threat-Assessment-System/blob/main/Dynamic_Threat_Assessment_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install newsapi-python

In [11]:
# Import NewsApiClient
from newsapi import NewsApiClient

# Initialize NewsAPI client with your API key
newsapi = NewsApiClient(api_key="72e3f1667346402d83d2a5eef280e38a")  # Replace with your valid API key

# Function to fetch news articles
def fetch_news_data(query, language="en", page_size=10):
    """
    Fetches news articles based on the given query using NewsAPI.

    Args:
        query (str): Search query for articles.
        language (str): Language of the articles (default is 'en').
        page_size (int): Number of articles to fetch (default is 10).

    Returns:
        list: A list of dictionaries containing news article details.
    """
    try:
        # Fetch articles from NewsAPI
        articles = newsapi.get_everything(q=query, language=language, page_size=page_size)
        # Process and return the articles
        return [
            {
                "title": article["title"],
                "description": article["description"],
                "url": article["url"],
                "source": article["source"]["name"]
            }
            for article in articles["articles"]
        ]
    except Exception as e:
        print(f"Error fetching NewsAPI data: {e}")
        return []

# Example query for news articles
query = "cybersecurity OR threats"
news_data = fetch_news_data(query)

# Display the results
print(f"Collected {len(news_data)} news articles:")
for i, article in enumerate(news_data, 1):
    print(f"\nArticle {i}:")
    print(f"Title: {article['title']}")
    print(f"Description: {article['description']}")
    print(f"Source: {article['source']}")
    print(f"URL: {article['url']}")


Collected 6 news articles:

Article 1:
Title: US Cyber Trust Mark launches as the Energy Star of smart home security
Description: The US launched a new Cyber Trust Mark label for smart home devices to assure consumers that internet-connected products meet cybersecurity standards.
Source: The Verge
URL: https://www.theverge.com/2025/1/7/24338168/us-cyber-trust-mark-smart-home-security

Article 2:
Title: The FCC’s Jessica Rosenworcel Isn’t Leaving Without a Fight
Description: As the US faces “the worst telecommunications hack in our nation’s history,” by China’s Salt Typhoon hackers, the outgoing FCC chair is determined to bolster network security if it’s the last thing she does.
Source: Wired
URL: https://www.wired.com/story/the-fccs-jessica-rosenworcel-isnt-leaving-without-a-fight/

Article 3:
Title: A New Jam-Packed Biden Executive Order Tackles Cybersecurity, AI, and More
Description: US president Joe Biden just issued a 40-page executive order that aims to bolster federal cybersecur